In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    datasets = ['train']
    for ds in datasets:
        image_path = os.path.join(os.getcwd(),'drive/MyDrive/train/India/annotations/xmls')
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('labels_{}.csv'.format(ds), index=None)
        print('Successfully converted xml to csv.')
        !cp labels_train.csv "/content/drive/MyDrive"


main()

Successfully converted xml to csv.


In [ ]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm import tqdm

In [ ]:
df = pd.read_csv('drive/MyDrive/dataframe.csv')
df.head()

AttributeError: ignored

In [ ]:
uniqueImages = open('uniqueImg.txt','w')

imgs = df['filename'].unique()
for im in tqdm(imgs):
  uniqueImages.write(str(im))
  uniqueImages.write('\n')
uniqueImages.close()

100%|██████████| 3785/3785 [00:00<00:00, 490512.61it/s]


In [ ]:

def getFileCounts(folder):
    files = os.listdir(folder)
    count = 0
    for file in files:
        count += 1
    return count

def removeUnwantedFiles(fileList,folder):
    #this function removes the files in fileList from folder:
    
    with open(fileList, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            print(fileName)
            path = folder + '/' + fileName
            os.remove(path)
        f.close()

def getFileName(folder):
    allImg = open('allImg.txt','w')
    files = os.listdir(folder)
    for file in files:
        allImg.write(str(file))
        allImg.write('\n')
    allImg.close()


def getOutliers(allFiles, uniqueFiles, folder):
    allImages = []
    imgToKeep = []
    imgToDel = []
    with open(allFiles, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            allImages.append(fileName)
        f.close()
    
    with open(uniqueFiles, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            imgToKeep.append(fileName)
        f.close()   

    for img in allImages:
        if img not in imgToKeep:
            imgToDel.append(img)
    print(len(imgToDel))

    fileDel = open('imgToDel.txt','w')
    files = os.listdir(folder)
    for img in imgToDel:
        fileDel.write(str(img))
        fileDel.write('\n')
    fileDel.close()

#converting csv format to yolo format:
def csv_to_yolo(df):
  print(os.getcwd())
  dir = 'drive/MyDrive/darknet/data/labels/'
  labelsUsed = []
  labelsRepeat = []
  #D10,D40,D00,D20,D44,D50
  classIndex = {'D10':0, 'D40':1, 'D00':2, 'D20':3, 'D44':4, 'D50':5, 'D11':6, 'D01':7, 'D43':8, 'D0w0':9}
  for i in range(len(df)):
    fName = df['filename'].values[i].split('.')[0]
    if fName not  in labelsUsed:
      label = open(dir + str(fName) + '.txt', 'w')
      class_ = classIndex[df['class'].values[i]]
      image_width = df['width'].values[i]
      image_height = df['height'].values[i]
      xmin = df['xmin'].values[i]
      ymin = df['ymin'].values[i]
      xmax = df['xmax'].values[i]
      ymax = df['ymax'].values[i]

      x_coord = (xmin + xmax) / 2 / image_width
      y_coord = (ymin + ymax) / 2 / image_height
      shape_width = (xmax - xmin) / image_width
      shape_height = (ymax - ymin) / image_height

      label.write(str(class_) + ' ' + str(x_coord) + ' ' + str(y_coord) + ' ' + str(shape_width) + ' ' + str(shape_height))
      label.write('\n')
      label.close()
      labelsUsed.append(fName)
    else:
      label = open(dir + str(fName) + '.txt', 'a')
      class_ = classIndex[df['class'].values[i]]
      image_width = df['width'].values[i]
      image_height = df['height'].values[i]
      xmin = df['xmin'].values[i]
      ymin = df['ymin'].values[i]
      xmax = df['xmax'].values[i]
      ymax = df['ymax'].values[i]

      x_coord = (xmin + xmax) / 2 / image_width
      y_coord = (ymin + ymax) / 2 / image_height
      shape_width = (xmax - xmin) / image_width
      shape_height = (ymax - ymin) / image_height

      label.write(str(class_) + ' ' + str(x_coord) + ' ' + str(y_coord) + ' ' + str(shape_width) + ' ' + str(shape_height))
      label.write('\n')
      #label.read()
      label.close()
      
  #return labelsUsed, labelsRepeat

def trainTestSplit(uniqueImg):
    #listing directories for all the images in a txt file for train and validation set: 
    listImages = []
    with open(uniqueImg, 'r') as f:
        for line in f:
            fileName = line.rstrip()
            listImages.append(fileName)
        f.close()
    #listImages = df['filename'].values
    totalImages = len(listImages)
    print(totalImages)
    trainImages = int(0.8*totalImages)
    valImages = totalImages - trainImages

    trainFile = open('train.txt','w')
    valFile = open('val.txt','w')

    dir = 'data/images/'
    for i in range(trainImages):
        trainFile.write(dir + listImages[i])
        trainFile.write('\n')

    for i in range(valImages):
        valFile.write(dir + listImages[trainImages+i])
        valFile.write('\n')

    trainFile.close()
    valFile.close()


#df = pd.read_csv('dataframe.csv')
csv_to_yolo(df)

#trainTestSplit('uniqueImg.txt')
#numImages = getFileCounts('darknet/data/images')
#numLabels = getFileCounts('darknet/data/labels')
#print('Total images : ',numImages)
#print('Total labels : ',numLabels)
#getFileName('drive/MyDrive/darknet/data/images')
#getOutliers('allImg.txt', 'uniqueImg.txt', 'drive/MyDrive/darknet/data/images')
#removeUnwantedFiles('imgToDel.txt', 'drive/MyDrive/darknet/data/images' )
#numImages = getFileCounts('drive/MyDrive/darknet/data/images')
#print('Total images : ',numImages)

/content
